<a href="https://colab.research.google.com/github/cerezamo/NLP_brouillon/blob/master/BERT_TD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 2 : Classification

## Get the data ready 



In [2]:
import spacy 
import numpy as np 
import pandas as pd 
import os 
os.getcwd()
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!pip install transformers

In [0]:
df=pd.read_csv('medium_df_deseq.csv',encoding='utf-8')
#J'ai du save en utf8 ? 

In [5]:
df.head()

,Unnamed: 0,Id,Titre,Type,Theme,Prenom,Nom,Fonction,Date,Tags,Texte,Lien,PRENOM,preusuel,sexe
0,10504,144565,"Déclaration de M. Jacques Chirac, Président de...",déclaration,International,Jacques,Chirac,NaN,1998-10-03T12:00:00Z,"Défense,Politique de la défense","Messieurs,Je suis heureux de vous saluer. Quan...",https://www.vie-publique.fr/discours/144565-de...,JACQUES,JACQUES,1
1,3304,196091,"Déclaration de M. Laurent Fabius, ministre des...",déclaration,Société,Laurent,Fabius,NaN,2015-08-26T12:00:00Z,"Environnement,Climat",Tout ce que nous avons entendu au cours de cet...,https://www.vie-publique.fr/discours/196091-de...,LAURENT,LAURENT,1
2,8816,128510,"Déclaration de M. François Hollande, premier s...",déclaration,Institutions,François,Hollande,NaN,2002-03-14T12:00:00Z,"Citoyenneté - Elections,Personnalité politique","Cher(e)s Ami(e)s, Cher(e)s Camarades, C'est av...",https://www.vie-publique.fr/discours/128510-de...,FRANÇOIS,FRANÇOIS,1
3,19491,140697,"Déclaration de M. Alain Lambert, ministre délé...",déclaration,International,Alain,Lambert,NaN,2003-06-11T12:00:00Z,"Relations internationales,Frontière","Mesdames,messieurs, C'est avec grand plaisir q...",https://www.vie-publique.fr/discours/140697-de...,ALAIN,ALAIN,1
4,6289,195589,"Déclaration de M. François Hollande, Président...",déclaration,International,François,Hollande,NaN,2015-07-03T12:00:00Z,"Relations internationales,Relations bilatérale...","Mesdames, Messieurs, bonsoir - j'allais dire b...",https://www.vie-publique.fr/discours/195589-de...,FRANÇOIS,FRANÇOIS,1


In [0]:
df=df.sample(frac=1)
df.sexe=df.sexe.replace(1,0)
df.sexe=df.sexe.replace(2,1)
df=df.reset_index()
df=df[['Texte','sexe']]
df.Texte = df.Texte.apply(lambda x : x.replace('\xa0','').replace('\x85','').replace('\x96','').replace('\x92',"'").replace('\x80','').replace("\x89'","'").replace("\'","'"))

In [7]:
df.head()

,Texte,sexe
0,"Monsieur le Président du Sénat, cher Jean-Pier...",1
1,"Mesdames, Messieurs,Bonsoir,Merci d'être là et...",0
2,"Monsieur l'ambassadeur,Monsieur le consul géné...",0
3,"Madame, Monsieur Le Ministre,Messieurs les Dép...",0
4,"Monsieur le président,Mesdames et Messieurs le...",0


Let's clean a bit our corpus. We will try to get rid of the first part which always starts with 'Mesdames et Messieurs ... blablabla" 


In [0]:
df.Texte = df.Texte.apply(lambda x : x.replace("\'","'"))

In [0]:

def cleanText(x):

    """
        Fonction permettant de nettoyer et de tokenizer un texte
    """
    import string
    x = x.replace('Monsieur le Président','').replace('Mesdames','').replace('Messieurs les Députés','').replace('Messieurs','').replace('Mme la présidente','')
    x = x.replace('Monsieur le Ministre','').replace('Monsieur','').replace('sous-préfets','').replace('Madame la Députée','')
    x = x.replace('  ','')
    x = x.replace("\'","'")
    return x
df.Texte = df.Texte.apply(cleanText)

In [10]:
df.Texte[4]

" le président, etles députés, le député,L'Europe vient en effet, à l'unanimité, de choisir Cadarache, en Provences-Alpes-Côte d'Azur, comme site d'implantation du projet ITER. Candidat de l'Europe, Cadarache porte en effet nos espoirs pour ce grand projet scientifique, premier grand projet mondial : plus de 10 milliards d'euros d'investissement, des milliers d'emplois et, pour nous tous, une mobilisation de la communauté scientifique exceptionnelle. Mon premier sentiment est un sentiment de gratitude vis-à-vis de l'attitude de nos amis espagnols qui ont su faire preuve du sens de l'intérêt général en faisant que l'Europe se rassemble sur un seul projet pour gagner la compétition mondiale.Mon deuxième sentiment de gratitude va à la Commission, qui a préparée l'ensemble de ce dispositif, au Commissaire Busquin et à tous ces qui ont fait l'évaluation et qui ont ainsi révélé les atouts de Cadarache. Je voudrais saluer la communauté scientifique, les chercheurs, les ingénieurs, les technic

In [0]:
df=df[['Texte','sexe']]

## Méthode du TD dans le cours pour le preprocessing du texte (utilisation de bert)


In [12]:
df.head(2)

,Texte,sexe
0,"du Sénat, cher Jean-Pierre, de l'ARF, messieu...",1
1,", ,Bonsoir,Merci d'être là et d'avoir répondu ...",0


In [0]:
df=df.rename(columns={'Texte':'sentence','sexe':'label'})

In [0]:
train_size = round(len(df)*0.8)
test_size = round(len(df)*0.2)
stop  = train_size+test_size

In [0]:
df_train = df[0:train_size]

In [0]:
df_test = df[train_size:stop]

In [0]:
df_train.to_csv('train.tsv',header=True,index=False,sep='\t')
df_test.to_csv('test.tsv',header=True,index=False,sep='\t')

In [18]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import BertTokenizer, AutoModel, AutoTokenizer
import pandas as pd
from torch.utils.data import DataLoader

class SSTDataset(Dataset):

    def __init__(self, filename, maxlen, model_name='bert-base-multilingual-cased'):

        #Store the contents of the file in a pandas dataframe
        self.df = pd.read_csv(filename, delimiter = '\t')

        #Initialize the BERT tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame
        sentence = self.df.loc[index, 'sentence']
        label = self.df.loc[index, 'label']

        #Preprocessing the text to be suitable for BERT
        tokens = self.tokenizer.tokenize(sentence) #Tokenize the sentence
        if self.tokenizer.cls_token is None:
          bos_token = self.tokenizer.bos_token
        else:
          bos_token = self.tokenizer.cls_token
          
        if self.tokenizer.sep_token is None:
          eos_token = self.tokenizer.eos_token
        else:
          eos_token = self.tokenizer.sep_token
        
        tokens = [bos_token] + tokens + [eos_token] #Insering the CLS and SEP token in the beginning and end of the sentence
        if len(tokens) < self.maxlen:
            tokens = tokens + [self.tokenizer.pad_token for _ in range(self.maxlen - len(tokens))] #Padding sentences
        else:
            tokens = tokens[:self.maxlen-1] + [eos_token] #Prunning the list to be of specified max length

        tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor = torch.tensor(tokens_ids) #Converting the list to a pytorch tensor
        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask, label

In [0]:
#Creating instances of training and validation set
train_set = SSTDataset(filename = 'train.tsv', maxlen = 500, model_name='bert-base-multilingual-cased')
val_set = SSTDataset(filename = 'test.tsv', maxlen = 500, model_name='bert-base-multilingual-cased')

#Creating intsances of training and validation dataloaders
train_loader = DataLoader(train_set, batch_size = 6, num_workers = 5)
val_loader = DataLoader(val_set, batch_size = 6, num_workers = 5)

In [19]:
train_set.

SyntaxError: ignored

In [20]:
# get the number of sentences
print(train_set.__len__(), "number of sentence in training set")
print(val_set.__len__(), "number of sentence in validation set")

4000 number of sentence in training set
1000 number of sentence in validation set


In [21]:
# get  tokenized sentence indexed by 1 
train_set.__getitem__(1) 

(tensor([   101,    117,    117,  30120,  11669,  10835,    117,  10734,  10598,
            172,    112,  11684,  10331,  10131,    172,    112,  13508, 108811,
          10138,    254,  11464,  77674,    119,  13796,  12556,  81216,  10291,
          19263,  10141,  71393,  61595,  10121,    178,    112,  11346,  10104,
          24931,  21702,  10129,    119,  52386,  97184,  62303,    254,    180,
            112,  50560,  10176,  42951,  89120,  10322,  20888,    180,    112,
          21334,  10104,  31119,  19047,  38585,  10121,  14128,  10109,  23024,
          14202,  10131,    171,    112,  10176,  10794,  10121,  10144,  38993,
          10107,  10794,  50520,  12028,  51357,  73762,  10943,    119,  13796,
          49301,  26044,  10138,  10110,  19319,  76970,  10260,  10231,  17305,
          10121,    180,    112,  10135,  12568,  96076,    172,    112,  22930,
            119,    140,    112,  10176,  12452,  10121,  10109,  10688,  14583,
          10257,  20268,  10

Classification model 

In [0]:
class Sequenceclassifier(nn.Module):

    def __init__(self, pretrained_model_name='bert-base-multilingual-cased'):
        super(Sequenceclassifier, self).__init__()
        
        #Loading Mask Language Model 
        self.encoder = AutoModel.from_pretrained(pretrained_model_name)
        #we append an extra layer for Classification (it will be randomly initialized)
        self.cls_layer = nn.Linear(self.encoder.pooler.dense.out_features, 1)

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        #Feeding the input to BERT model to obtain contextualized representations
        cont_reps, _ = self.encoder(seq, attention_mask = attn_masks)

        #Obtaining the representation of [CLS] head
        cls_rep = cont_reps[:, 0]

        #Feeding cls_rep to the classifier layer
        logits = self.cls_layer(cls_rep)

        return logits

In [0]:
classification_model = Sequenceclassifier('bert-base-multilingual-cased')
# if gpu mode
classification_model = classification_model.to("cuda")
# to check if the weights of the model are in gpu : 
# sentiment_model.cls_layer.weight.is_cuda
# can checkout all the layers by running 
#sentiment_model

Defining the training process 



In [0]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
opti = optim.Adam(classification_model.parameters(), lr = 1e-3)

In [0]:
#classification_model.train()

In [0]:
import time


In [0]:
import pdb
def train(model, criterion, opti, train_loader, val_loader, max_eps=1, gpu=False, print_every=1,validate_every=1):
    if gpu:
      model = model.to("cuda")
    for ep in range(max_eps):
        
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            if gpu:
              seq, attn_masks, labels = seq.cuda(), attn_masks.cuda(), labels.cuda()
            #Obtaining the logits from the model
            logits = model(seq, attn_masks)

            #Computing loss
            loss = criterion(logits.squeeze(-1), labels.float())

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            opti.step()
            if (it + 1) % print_every == 0:
                accuracy = torch.sum((logits>0).int().squeeze(1)==labels)/float(labels.size(0))
                print("Iteration {} of epoch {} complete. Loss : {}, Accuracy {} ".format(it+1, ep+1, loss.item(),accuracy))
            if it>1000:
              break
        if ep % validate_every==0:
          n_batch_validation = 0
          loss_validation = 0
          accuracy_validation = 0
          for it, (seq, attn_masks, labels) in enumerate(val_loader):
            #Clear gradients
            
            if gpu:
              seq, attn_masks, labels = seq.cuda(), attn_masks.cuda(), labels.cuda()
            #Obtaining the logits from the model
            logits_val = model(seq, attn_masks)
            n_batch_validation+=1
            #Computing loss
           
            _loss = float(criterion(logits_val.squeeze(-1), labels.float()))
            _accu = float(torch.sum((logits_val>0).int().squeeze(1)==labels)/float(labels.size(0)))
           
            loss_validation += _loss
            accuracy_validation += _accu
          print("EVALUATION Validation set : mean loss {} n mean accuracy {}".format(loss_validation/n_batch_validation, accuracy_validation/n_batch_validation))

In [40]:
train(classification_model, criterion, opti, train_loader, val_loader, max_eps=1, print_every=100, gpu=True)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
EVALUATION Validation set : mean loss 0.5295566314575719 n mean accuracy 0.7777778003730026
EVALUATION Validation set : mean loss 0.5279688313603401 n mean accuracy 0.7788461767710172
EVALUATION Validation set : mean loss 0.5306096015111456 n mean accuracy 0.7767295826156184
EVALUATION Validation set : mean loss 0.5332462064645909 n mean accuracy 0.7746913808363455
EVALUATION Validation set : mean loss 0.5318858385086059 n mean accuracy 0.7757575988769532
EVALUATION Validation set : mean loss 0.5271229264991624 n mean accuracy 0.7797619274684361
EVALUATION Validation set : mean loss 0.52607370730032 n mean accuracy 0.7807017773912664
EVALUATION Validation set : mean loss 0.532156545026549 n mean accuracy 0.7758620915741756
EVALUATION Validation set : mean loss 0.5277391429674827 n mean accuracy 0.7796610391746133
EVALUATION Validation set : mean loss 0.5300069928169251 n mean accuracy 0.7777777999639511
EVALU

KeyboardInterrupt: ignored

NameError: ignored

In [0]:
with torch.no_grad():
    outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    predictions = outputs[0]

In [96]:
import tensorflow as tf
# GPU a activer 
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [10]:
device = torch.device("cuda")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [0]:
import transformers

In [63]:
df.Texte[0]

"Mme la présidente. L'ordre du jour appelle le débat sur la judiciarisation et la criminalisation de l'action militante. La conférence des présidents a décidé d'organiser ce débat en deux parties. Dans un premier temps, nous entendrons les orateurs des groupes, puis le Gouvernement. Nous procéderons ensuite à une séquence de questions et réponses. Je vous rappelle que la durée des questions, ainsi que celle des réponses, est limitée à deux minutes, sans droit de réplique. Je vous demande de respecter ce temps de parole. (…) Mme la présidente. La parole est à Mme la garde des sceaux, ministre de la justice. Mme Nicole Belloubet, garde des sceaux, ministre de la justice. Mes premiers mots iront à M. le député Naegelen, dont les propos rejoignent mes pensées s'agissant des surveillants pénitentiaires qui ont été gravement blessés à la suite du très lâche attentat terroriste qui s'est déroulé à la prison de Condé-sur-Sarthe. Je vous remercie, monsieur le député. S'agissant du débat qui nou

In [0]:
df.Texte = df.Texte.apply(lambda x : x.replace('Monsieur le Président','').replace('Mesdames','').replace('Messieurs les Députés','').replace('Messieurs','').replace('Mme la présidente',''))

In [0]:
df.Texte = df.Texte.apply(lambda x : x.replace('Monsieur le Ministre','').replace('Monsieur',''))

In [0]:
df.Texte = df.Texte.apply(lambda x : x.replace("\'","'"))

In [73]:
df.Texte[10]

" les Députées, Laure de la RAUDIERE et Corinne ERHEL,, Jean-Ludovic SILICANI, et , Je voudrais tout d'abord vous remercier, Madame la députée Laure de la RAUDIERE pour votre invitation à inaugurer ces 5èmes Assises du très haut débit. Le déploiement du très haut débit représente un enjeu industriel majeur pour notre pays, un levier pour la compétitivité de nos entreprises et un facteur essentiel d'aménagement de nos territoires. Depuis l'adoption du plan France numérique 2012, le déploiement du très haut débit est une composante fondamentale de notre politique numérique. Je souhaiterais vous en présenter trois priorités. Premièrement, le développement de l'Internet très haut débit est conditionné à la disponibilité de ressources suffisantes. Le système d'adressage IPv4 est aujourd'hui saturé et pourrait freiner le développement de nouveaux services. La première « journée mondiale IPv6 » s'est déroulée hier. Elle a permis de conclure à un niveau insuffisant de préparation dans la trans

In [0]:
import pdb
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from transformers import XLMRobertaForMaskedLM, XLMRobertaTokenizer

In [78]:
!nvidia-smi

Wed Mar 25 15:07:33 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    32W / 250W |    717MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from transformers import CamembertTokenizer

In [0]:
camembert = CamembertModel.from_pretrained('camembert-base')

In [0]:
sentences = df.Texte.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + ' ' + sentence + " [SEP]" for sentence in sentences]
labels = df.sexe.values

In [89]:
tokenizer = CamembertTokenizer.from_pretrained('camembert-base', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

Tokenize the first sentence:
['▁[', 'cl', 's', ']', '▁', '.', '▁l', "'", 'ordre', '▁du', '▁jour', '▁appelle', '▁le', '▁débat', '▁sur', '▁la', '▁j', 'udi', 'cia', 'risation', '▁et', '▁la', '▁c', 'riminal', 'isation', '▁de', '▁l', "'", 'action', '▁militante', '.', '▁la', '▁conférence', '▁des', '▁président', 's', '▁a', '▁décidé', '▁d', "'", 'organiser', '▁ce', '▁débat', '▁en', '▁deux', '▁parties', '.', '▁dans', '▁un', '▁premier', '▁temps', ',', '▁nous', '▁entend', 'rons', '▁les', '▁or', 'ateurs', '▁des', '▁groupes', ',', '▁puis', '▁le', '▁gouvernement', '.', '▁nous', '▁procéder', 'ons', '▁ensuite', '▁à', '▁une', '▁séquence', '▁de', '▁questions', '▁et', '▁réponses', '.', '▁je', '▁vous', '▁rappelle', '▁que', '▁la', '▁durée', '▁des', '▁questions', ',', '▁ainsi', '▁que', '▁celle', '▁des', '▁réponses', ',', '▁est', '▁limitée', '▁à', '▁deux', '▁minutes', ',', '▁sans', '▁droit', '▁de', '▁réplique', '.', '▁je', '▁vous', '▁demande', '▁de', '▁respecter', '▁ce', '▁temps', '▁de', '▁parole', '.', '▁(.

In [87]:
len(max(tokenized_texts, key=len)),len(min(tokenized_texts, key=len))

NameError: ignored

In [0]:
MAX_LEN = 512

In [0]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [0]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [0]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)
  

In [0]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [0]:

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import BertTokenizer, AutoModel, AutoTokenizer
import pandas as pd
from torch.utils.data import DataLoader

class SSTDataset(Dataset):

    def __init__(self, filename, maxlen, model_name='bert-base-uncased'):

        #Store the contents of the file in a pandas dataframe
        self.df = pd.read_csv(filename, delimiter = '\t')

        #Initialize the BERT tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame
        sentence = self.df.loc[index, 'sentence']
        label = self.df.loc[index, 'label']

        #Preprocessing the text to be suitable for BERT
        tokens = self.tokenizer.tokenize(sentence) #Tokenize the sentence
        if self.tokenizer.cls_token is None:
          bos_token = self.tokenizer.bos_token
        else:
          bos_token = self.tokenizer.cls_token
          
        if self.tokenizer.sep_token is None:
          eos_token = self.tokenizer.eos_token
        else:
          eos_token = self.tokenizer.sep_token
        
        tokens = [bos_token] + tokens + [eos_token] #Insering the CLS and SEP token in the beginning and end of the sentence
        if len(tokens) < self.maxlen:
            tokens = tokens + [self.tokenizer.pad_token for _ in range(self.maxlen - len(tokens))] #Padding sentences
        else:
            tokens = tokens[:self.maxlen-1] + [eos_token] #Prunning the list to be of specified max length

        tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor = torch.tensor(tokens_ids) #Converting the list to a pytorch tensor
        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask, label

In [0]:
from transformers import CamembertModel

In [75]:
camembert = CamembertModel.from_pretrained('camembert-base')

In [0]:
sentences = df.Texte.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + ' ' + sentence + " [SEP]" for sentence in sentences]
labels = df.sexe.values

In [76]:
sentences[0]

NameError: ignored